In [1]:
# fix seeds 
SEED = 42

import random as rn
rn.seed(SEED)

import numpy as np
np.random.seed(SEED)

import tensorflow as tf
tf.random.set_seed(SEED)

import sys
sys.path.append("..")

In [2]:
from src.dataset.big_earth_dataset import BigEarthDataset
from src.dataset.segmentation_dataset import SegmentationDataset
# big_earth = BigEarthDataset('/workspace/data/prepped/dev-train.tfrecord')
# train_dataset = big_earth.dataset
# val_dataset = BigEarthDataset('/workspace/data/prepped/dev-test.tfrecord', shuffle_buffer_size=0, batch_size=1).dataset

#segmentation = SegmentationDataset('/workspace/data/prepped/segmentation-train-5000.tfrecord').dataset
d = SegmentationDataset('/workspace/data/prepped/segmentation-val-dev.tfrecord').dataset
for batch in d:
    for item in batch:
        print(item.shape)


DataLossError: corrupted record at 0

In [8]:
from src.model.unet import Unet
import segmentation_models as sm

model = Unet(
    input_shape=(128, 128, 10),
    classes=5,  
    encoder_weights='test-weights.h5',
    encoder_freeze=True,
)
model.compile(
    optimizer='adam',
    loss=sm.losses.categorical_focal_loss,
    metrics=[
        'accuracy', 
        sm.metrics.iou_score,
    ], 
)
model.fit(
    segmentation, 
    epochs=1,
    validation_data=val_dataset,
    validation_steps=2,
    callbacks=[
        tf.keras.callbacks.CSVLogger('logs/metrics.csv'), 
        tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5), 
        tf.keras.callbacks.ReduceLROnPlateau(), 
        tf.keras.callbacks.TensorBoard('logs/test', profile_batch=0)  
    ],
    workers=4,
    use_multiprocessing=True
)

1/1 [==============================] - 25s 25s/step - loss: 0.0685 - accuracy: 0.1391 - iou_score: 0.0557 - val_loss: 0.7755 - val_accuracy: 0.0061 - val_iou_score: 0.0014 - lr: 0.0010


In [ ]:
# results = model.evaluate(train_dataset)
# print("Loss :", results[0])
# print("Accuracy :", results[1])
#preds = model.predict(train_dataset)
#model.summary()

features = model.get_layer('decoder_stage4b_relu').output
cam = model.get_layer('final_fc')(features)